## Data loading

In [13]:
import pandas as pd

In [14]:
path = "E:\\Nikhil\\VScode\\ML\\datasets&models\\"

In [16]:
df = pd.read_json(path + "Gpt&human_data.jsonl", lines=True)

In [17]:
df.shape

(24322, 5)

In [18]:
df.head()

,question,human_answers,chatgpt_answers,index,source
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN,reddit_eli5
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN,reddit_eli5
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN,reddit_eli5
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN,reddit_eli5
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN,reddit_eli5


In [19]:
df.drop(["index","source"],axis=1, inplace=True)
df.head()

,question,human_answers,chatgpt_answers
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...


In [20]:
print(df["chatgpt_answers"].isna().sum())
print(df["human_answers"].isna().sum())

0
0


In [21]:
df["human_answers"][:1][0]

['Basically there are many categories of " Best Seller " . Replace " Best Seller " by something like " Oscars " and every " best seller " book is basically an " oscar - winning " book . May not have won the " Best film " , but even if you won the best director or best script , you \'re still an " oscar - winning " film . Same thing for best sellers . Also , IIRC the rankings change every week or something like that . Some you might not be best seller one week , but you may be the next week . I guess even if you do n\'t stay there for long , you still achieved the status . Hence , # 1 best seller .',
 "If you 're hearing about it , it 's because it was a very good or very well - publicized book ( or both ) , and almost every good or well - publicized book will be # 1 on the NY Times bestseller list for at least a little bit . Kindof like how almost every big or good movies are # 1 at the box office on their opening weekend .",
 "One reason is lots of catagories . However , how the NY Ti

In [22]:
text = df["chatgpt_answers"][:1][0]
text

['There are many different best seller lists that are published by various organizations, and the New York Times is just one of them. The New York Times best seller list is a weekly list that ranks the best-selling books in the United States based on sales data from a number of different retailers. The list is published in the New York Times newspaper and is widely considered to be one of the most influential best seller lists in the book industry. \nIt\'s important to note that the New York Times best seller list is not the only best seller list out there, and there are many other lists that rank the top-selling books in different categories or in different countries. So it\'s possible that a book could be a best seller on one list but not on another. \nAdditionally, the term "best seller" is often used more broadly to refer to any book that is selling well, regardless of whether it is on a specific best seller list or not. So it\'s possible that you may hear about a book being a "bes

## data Preprocess

In [53]:
import re
import spacy
nlp = spacy.load("en_core_web_lg")

In [54]:
def preprocess(data):
    data = str(data)
    data = data.lower()
    data = re.sub(r"[^0-9a-zA-Z]", " ",data)
    data = re.sub(r" +", " ", data)
    data = data.strip()

    filtered_token =[]
    doc = nlp(data)
    for token in doc:
        if token is token.is_punct or token.is_stop:
            continue
        filtered_token.append(token.lemma_)

    return " ".join(filtered_token)

In [34]:
preprocess(str(text))

'different good seller list publish organization new york times new york times good seller list weekly list rank good selling book united states base sale datum number different retailer list publish new york times newspaper widely consider influential good seller list book industry nit s important note new york times good seller list good seller list list rank sell book different category different country s possible book good seller list nadditionally term good seller broadly refer book sell regardless specific good seller list s possible hear book good seller specifically rank number good seller new york times list list'

In [35]:
df["chatgpt_answers"].shape

(24322,)

In [17]:
%%time
df["cleaned_gpt_ans"] = df["chatgpt_answers"].apply(preprocess)

In [19]:
%%time
df["cleaned_human_ans"] = df["human_answers"].apply(preprocess)

CPU times: total: 25min 4s
Wall time: 35min 59s


## Formatting data for training

In [20]:
df["cleaned_gpt_ans"] = "__label__AIGenerated" + " " + df["cleaned_gpt_ans"]
df["cleaned_human_ans"] = "__label__HumanWritten" + " " + df["cleaned_human_ans"]

In [21]:
df.head()

,question,human_answers,chatgpt_answers,cleaned_gpt_ans,cleaned_human_ans
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,__label__AIGenerated different best seller lis...,__label__HumanWritten basically categories bes...
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,__label__AIGenerated salt roads help melt ice ...,__label__HumanWritten salt good dying car cras...
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,__label__AIGenerated reasons sd standard defin...,__label__HumanWritten way works old tv station...
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,__label__AIGenerated generally acceptable ethi...,__label__HumanWritten n t assassinating leader...
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,__label__AIGenerated wright brothers powered f...,__label__HumanWritten wanting kill shit german...


In [26]:
df2 = pd.concat([df["cleaned_gpt_ans"],df["cleaned_human_ans"]], ignore_index=True,axis=0)

In [27]:
df2.shape

(48644,)

In [28]:
df2

0        __label__AIGenerated different best seller lis...
1        __label__AIGenerated salt roads help melt ice ...
2        __label__AIGenerated reasons sd standard defin...
3        __label__AIGenerated generally acceptable ethi...
4        __label__AIGenerated wright brothers powered f...
                               ...                        
48639    __label__HumanWritten hello welcome thank aski...
48640    __label__HumanWritten hi surgical experience i...
48641    __label__HumanWritten difficult comment acutre...
48642    __label__HumanWritten welcome thank asking hcm...
48643    __label__HumanWritten hi having type pain age ...
Length: 48644, dtype: object

In [29]:
df2.to_csv("plagarism_check_data.txt",index=False, header=False)

In [36]:
import fasttext

### training model on formated data

In [50]:
%%time
#model = fasttext.train_supervised("plagarism_check_data.txt", ws=5, lr=0.05, epoch=50, dim=300, minn=2, maxn=7)

CPU times: total: 3h 44min 56s
Wall time: 21min 27s


### save model for further application

In [51]:
#model.save_model("fasttext_plagarism3.model")

In [41]:
path = "E:\\Nikhil\\VScode\\ML\\datasets&models\\"

### load saved model

In [55]:
model = fasttext.load_model(path + "fasttext_plagarism3.model")

In [43]:
model.get_word_vector("machine").shape

(300,)

In [44]:
model.get_nearest_neighbors("machine")

[(0.9921006560325623, 'machines'),
 (0.9898130893707275, 'machins'),
 (0.9895593523979187, 'submachine'),
 (0.9884148836135864, 'machin'),
 (0.9872796535491943, 'machined'),
 (0.9854716062545776, 'nmachine'),
 (0.9853755235671997, 'nanomachines'),
 (0.9847303628921509, 'nengine'),
 (0.9846785068511963, 'joachin'),
 (0.9844298958778381, 'npowerline')]

In [45]:
prediction = model.predict(preprocess("""ohn left me, and a few minutes later I saw him from my window walking slowly across the grass arm in arm with Cynthia Murdoch. I heard Mrs. Inglethorp call "Cynthia" impatiently, and the girl started and ran back to the house."""))

In [47]:
prediction

(('__label__HumanWritten',), array([0.99998307]))

## prediction fun

In [56]:
def predict(text):
    text = preprocess(text)
    prediction = model.predict(text)

    if prediction[0][0]=="__label__HumanWritten":
        print(f"{(100 - prediction[1][0]*100).round(2)}% plagiarism detected")

    else:
        print(f"{(prediction[1][0]*100).round(2)}% plagiarism detected")

In [57]:
human_written = "The aroma of warm bread and buttery croissants filled the air, pulling Amelia towards the bakery like a magnet. Stepping inside, the cozy atmosphere embraced her. Flour dusted baskets overflowed with golden baguettes, and a chalkboard menu scrawled enticing specials. Amelia's stomach rumbled, and a smile crept across her face. Today, she was indulging."
AI_generated = "Bakery X offers a variety of breads and pastries made with fresh, high-quality ingredients. Their menu includes baguettes, croissants, muffins, and more. They also offer a selection of sandwiches and coffee drinks."

predict(human_written)
predict(AI_generated)

3.46% plagiarism detected
99.98% plagiarism detected


In [63]:
predict("""
 🚀Excited to share my latest project on Plagiarism Detection using NLP! 🚀

Over the past three weeks, I've dived deep into Natural Language Processing (NLP) and honed my skills in tools like NLTK, FastText, Gensim, and spaCy. This journey has culminated in the creation of a plagiarism detection system, leveraging word embeddings in Gensim and advanced preprocessing techniques.

🔍 Project Highlights:

📝Utilized word embeddings in Gensim for accurate text similarity measurement.

📝Implemented comprehensive text preprocessing techniques including tokenization, lemmatization, and stop-word removal.

📝Integrated multiple NLP tools such as NLTK and spaCy for efficient text processing.

This project has not only deepened my understanding of NLP but also opened up new avenues for future exploration.

#NLP #MachineLearning #ArtificialIntelligence #PlagiarismDetection #Gensim #NLTK #spaCy #FastText #DataScience #Tech
""")

64.05% plagiarism detected
